In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [857 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,102 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [832 kB]
Get:13 http://security.ubunt

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [10]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
view_name = "home_sales_view"
df.createOrReplaceTempView(view_name)
df_head = spark.sql(f"SELECT * FROM home_sales_view LIMIT 1")
df_head.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+



In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT YEAR(date_built) AS year,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_view
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""

result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|296800.75|
|2011| 302141.9|
|2012|298233.42|
|2013|299999.39|
|2014|299073.89|
|2015|307908.86|
|2016|296050.24|
|2017|296576.69|
+----+---------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT YEAR(date_built) AS year,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_view
WHERE bedrooms = 3
AND bathrooms = 3
GROUP BY year
ORDER BY year
"""
result = spark.sql(query)
result.show()

+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT YEAR(date_built) AS year,
       ROUND(AVG(price), 2) AS avg_price
FROM home_sales_view
WHERE bedrooms = 3
AND bathrooms = 3
AND floors = 2
AND sqft_living > 2000
GROUP BY year
ORDER BY year
"""
result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|297619.46|
|2015|297609.97|
|2016| 293965.1|
|2017|280527.62|
+----+---------+



In [14]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
SELECT ROUND(AVG(price), 2) AS avg_price
FROM home_sales_view
WHERE price >= 350000
"""

start_time = time.time()

result = spark.sql(query)
average_price = result.collect()[0]["avg_price"]

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.6104879379272461 seconds ---


In [15]:
# 7. Cache the the temporary table home_sales.
spark.sql(f"CACHE TABLE home_sales_view")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [17]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT ROUND(AVG(price), 2) AS avg_price
FROM home_sales_view
WHERE price >= 350000
"""

start_time = time.time()

result = spark.sql(query)
average_price = result.collect()[0]["avg_price"]

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.3403451442718506 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("partitioned_home_sales.parquet")

In [19]:
# 11. Read the parquet formatted data.
spark = SparkSession.builder.appName("ParquetReader").getOrCreate()

# Read the partitioned Parquet data
parquet_path = "partitioned_home_sales.parquet"
partitioned_df = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame
partitioned_df.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [21]:
# 12. Create a temporary table for the parquet data.
table_name = "parquet_home_sales"
partitioned_df.createOrReplaceTempView(table_name)

# Now you can use the temporary table for querying with Spark SQL
result = spark.sql("SELECT * FROM parquet_home_sales LIMIT 1")
result.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+



In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query = """
SELECT ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
WHERE price >= 350000
"""

result = spark.sql(query)
average_price = result.collect()[0]["avg_price"]

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.43680715560913086 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.
spark = SparkSession.builder.appName("UncacheTable").getOrCreate()

# Uncache the temporary table
table_name = "home_sales_view"
spark.catalog.uncacheTable(table_name)

In [25]:
# 15. Check if the home_sales is no longer cached
table_name = "home_sales_view"
is_cached = spark.catalog.isCached(table_name)
is_cached


False